In [1]:
from nba_api.stats.endpoints import playergamelog, leaguegamelog, commonplayerinfo
from nba_api.stats.static import players
from nba_api.stats.library.parameters import SeasonType
import pandas as pd
import time

In [2]:
#We need a separate call for playoffs and regular season
hist_player_log = playergamelog.PlayerGameLog(player_id='202695',
                                              date_from_nullable='10/01/2000', date_to_nullable='01/11/2019'
                                              , season_all='ALL'
                                              , season_type_all_star='Regular Season').get_data_frames()[0]
hist_player_log.head()

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22018,202695,0021800624,"JAN 11, 2019",TOR vs. BKN,W,27,9,18,0.500,...,7,11,4,1,0,2,1,20,24,1
1,22018,202695,0021800602,"JAN 08, 2019",TOR vs. ATL,W,35,11,18,0.611,...,3,4,6,6,1,1,2,31,10,1
2,22018,202695,0021800580,"JAN 05, 2019",TOR @ MIL,W,38,8,16,0.500,...,7,7,6,5,0,2,1,30,21,1
3,22018,202695,0021800563,"JAN 03, 2019",TOR @ SAS,L,33,8,13,0.615,...,1,1,5,2,0,3,2,21,-22,1
4,22018,202695,0021800549,"JAN 01, 2019",TOR vs. UTA,W,35,16,22,0.727,...,6,6,1,0,0,2,2,45,8,1


In [3]:
hist_player_log_playoffs = playergamelog.PlayerGameLog(player_id='202695',
                                              date_from_nullable='10/01/2000', date_to_nullable='01/11/2019'
                                              , season_all='ALL'
                                              , season_type_all_star='Playoffs').get_data_frames()[0]
hist_player_log_playoffs.head()

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,42016,202695,0041600311,"MAY 14, 2017",SAS @ GSW,L,24,7,13,0.538,...,4,8,3,1,0,1,2,26,21,1
1,42016,202695,0041600235,"MAY 09, 2017",SAS vs. HOU,W,38,8,21,0.381,...,12,15,4,2,2,2,1,22,-2,1
2,42016,202695,0041600234,"MAY 07, 2017",SAS @ HOU,L,30,7,14,0.500,...,4,6,4,1,1,3,2,16,-15,1
3,42016,202695,0041600233,"MAY 05, 2017",SAS @ HOU,W,41,9,20,0.450,...,7,10,7,1,0,3,4,26,10,1
4,42016,202695,0041600232,"MAY 03, 2017",SAS vs. HOU,W,38,13,16,0.813,...,6,7,8,3,1,2,0,34,24,1


In [2]:
nba_players = players.get_players()
print('Number of players fetched: {}'.format(len(nba_players)))

Number of players fetched: 4374


In [ ]:
#Grab all player logs from Tuesday, October 27, 2015 (start of 15-16 season)
hist_player_log_full = pd.DataFrame()
nba_players[0]['full_name']
for i in range(0, len(nba_players)):
    player_id = nba_players[i]['id']
    hist_player_log = playergamelog.PlayerGameLog(player_id=player_id, date_from_nullable='10/27/2015'
                                                  , date_to_nullable='01/11/2019', season_all='ALL'
                                                  , season_type_all_star='Regular Season').get_data_frames()[0]
    hist_player_log['full_name'] = nba_players[i]['full_name']
    hist_player_log_full = hist_player_log_full.append(hist_player_log, ignore_index=True)
    time.sleep(0.5)
   

In [53]:
#Grab these players playoff game logs, if they exist
for i in hist_player_log_full['Player_ID'].unique():
    player_id = i
    hist_player_log = playergamelog.PlayerGameLog(player_id=player_id, date_from_nullable='10/27/2015'
                                                  , date_to_nullable='01/11/2019', season_all='ALL'
                                                  , season_type_all_star='Playoffs').get_data_frames()[0]
    hist_player_log['full_name'] = hist_player_log_full['full_name'][hist_player_log_full['Player_ID']==i].unique()[0]
    hist_player_log_full = hist_player_log_full.append(hist_player_log, ignore_index=True)
    time.sleep(0.5)

In [5]:
# #Grab the league start dates and playoff end dates
playergamelog.PlayerGameLog(player_id=202695, date_from_nullable='10/27/2015'
                                                  , date_to_nullable='01/11/2019', season_all='ALL'
                                                  , season_type_all_star='Playoffs').get_data_frames()[0]
all_playoff_games = leaguegamelog.LeagueGameLog(season_type_all_star='Playoffs', season_all_time='ALLTIME') \
                        .get_data_frames()[0]
#last game of the complete season
last_games = all_playoff_games.groupby('SEASON_ID').agg({'GAME_DATE' : 'max'})
all_season_games = leaguegamelog.LeagueGameLog(season_type_all_star='Regular Season', date_from_nullable='10/27/2015'
                                               , date_to_nullable='01/11/2019', season_all_time='ALLTIME') \
                        .get_data_frames()[0]
first_games = all_season_games.groupby('SEASON_ID').agg({'GAME_DATE' : 'min'})

In [7]:
first_games

,GAME_DATE
SEASON_ID,
22015,2015-10-27
22016,2016-10-25
22017,2017-10-17
22018,2018-10-16


In [10]:
#Grab player info (date of birth, draft year, etc.)
hist_player_info_full = pd.DataFrame()
for i in hist_player_log_full['Player_ID'].unique():
    player_id = i
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id).get_data_frames()[0][
        ['PERSON_ID', 'BIRTHDATE', 'FROM_YEAR','HEIGHT', 'WEIGHT', 'POSITION']]
    player_info['full_name'] = hist_player_log_full['full_name'][hist_player_log_full['Player_ID']==i].unique()[0]
    hist_player_info_full = hist_player_info_full.append(player_info, ignore_index=True)
    time.sleep(0.25)

In [84]:
hist_player_info_full.head()

,PERSON_ID,BIRTHDATE,FROM_YEAR,HEIGHT,WEIGHT,POSITION,full_name
0,203518,1993-08-01T00:00:00,2016,,,,Alex Abrines
1,203112,1990-10-06T00:00:00,2012,,,,Quincy Acy
2,1629121,1996-05-04T00:00:00,2018,74,190,Guard,Jaylen Adams
3,203919,1994-07-08T00:00:00,2014,77,209,Guard,Jordan Adams
4,203500,1993-07-20T00:00:00,2013,84,265,Center,Steven Adams


In [83]:
#Convert player height to inches
hist_player_info_full['HEIGHT'] = hist_player_info_full['HEIGHT'].apply(
    lambda x: float(str(x).split("-")[0]) * 12 + float(str(x).split("-")[1]) if len(str(x).split("-"))==2
    else ((str(x).split("-")[0]) * 12 if len(str(x).split("-"))==1 else x))

In [85]:
hist_player_info_full.to_csv('hist_player_info.csv', index=False)

In [87]:
#Merge with dataset
hist_player_log_full = hist_player_log_full.merge(hist_player_info_full[['PERSON_ID', 'BIRTHDATE', 'FROM_YEAR', 'HEIGHT', 'WEIGHT', 'POSITION']].drop_duplicates()
                           , left_on='Player_ID' , right_on='PERSON_ID', how='inner')

In [90]:
# hist_player_log_full.to_csv('hist_player_log.csv', index=False)

In [91]:
hist_player_log_full = pd.read_csv('hist_player_log.csv')
len(hist_player_log_full)

97014

In [92]:
hist_player_log_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97014 entries, 0 to 97013
Data columns (total 34 columns):
SEASON_ID          97014 non-null int64
Player_ID          97014 non-null int64
Game_ID            97014 non-null int64
GAME_DATE          97014 non-null object
MATCHUP            97014 non-null object
WL                 97014 non-null object
MIN                97014 non-null int64
FGM                97014 non-null int64
FGA                97014 non-null int64
FG_PCT             97014 non-null float64
FG3M               97014 non-null int64
FG3A               97014 non-null int64
FG3_PCT            97014 non-null float64
FTM                97014 non-null int64
FTA                97014 non-null int64
FT_PCT             97014 non-null float64
OREB               97014 non-null int64
DREB               97014 non-null int64
REB                97014 non-null int64
AST                97014 non-null int64
STL                97014 non-null int64
BLK                97014 non-null int64
TO

In [93]:
hist_player_log_full[(hist_player_log_full['full_name']=='Kawhi Leonard') & (hist_player_log_full['GAME_DATE']=='MAY 14, 2017')]

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,PTS,PLUS_MINUS,VIDEO_AVAILABLE,full_name,PERSON_ID,BIRTHDATE,FROM_YEAR,HEIGHT,WEIGHT,POSITION
54376,42016,202695,41600311,"MAY 14, 2017",SAS @ GSW,L,24,7,13,0.538,...,26,21,1,Kawhi Leonard,202695,1991-06-29T00:00:00,2011,79.0,230.0,Forward
